# <font color=#cd0000> Propósito principal </font>
- La idea de este librillo es preparar el entorno para realizar pruebas para cualquier DataSet a ser clasificado por cualquier codificación aplicada a RF

## <font color=#cd0000> Leyenda </font>
- Los apartados titulados con el código de colores HEX: `#cd0000` serán apartados que tendrán todos los librillos, en concreto, aquellos especificados en el apartado `Síntesis de los criterios usados` del trabajo.
- Los apartados titulados con el código de colores HEX: `#2451ff` serán apartados de conclusiones propias de este librillo resultado de aplicar un estudio personalizado para cada planteamiento.

# <font color=#cd0000> Prerrequisitos </font>
## <font color=#cd0000> Entorno de ejecución </font>
- Cambiamos el directorio raíz del librillo para acceder cómodamente a las funciones de utilidad.

In [ ]:
import os

os.chdir('../..')
os.listdir()


## <font color=#cd0000> Constantes y variables predefinidas </font>

In [ ]:
HEARTBEAT_PATH = "data/heartbeat"
EPILEPSY_PATH = "data/epilepsy"
SEGUIMIENTO_OCULAR_PATH = "data/seguimiento-ocular/Data/Hospital"
SEGUIMIENTO_OCULAR_FOLDERS_ID = range(1, 12+1)

DATA_TO_SAVE = "HeartBeat"

PKL_DIR = "pkl/<classifier>/<ds>/"
PKL_NAME = "<ds>_<classifier>_<codif>.pkl"

SEED = 1


# <font color=#cd0000> Carga del Dataset </font>

In [ ]:
# TODO - Change with known data
from utils.load_data import import_epilepsy_dataset

# train, test = import_heartbeat_dataset(HEARTBEAT_PATH)
# all_data = import_seguimiento_ocular_dataset(SEGUIMIENTO_OCULAR_PATH)


In [ ]:
# import pickle

# pickle.dump(train, open(DATA_TO_SAVE + "_tmp_train_data.pkl", 'wb'))
# pickle.dump(test, open(DATA_TO_SAVE + "_tmp_test_data.pkl", 'wb'))


In [ ]:
# import pickle

# train = pickle.load(open(DATA_TO_SAVE + "_tmp_train_data.pkl", 'rb'))
# test = pickle.load(open(DATA_TO_SAVE + "_tmp_test_data.pkl", 'rb'))


In [ ]:
train.reset_changes()
test.reset_changes()


## <font color=#cd0000> Particionado inicial de los datos si fuera necesario </font>

In [ ]:
# from utils.data_extraction import Data

# X_train_Data, X_test_Data, y_train, y_test = all_data.train_test_split(
#     criterion='windowed',
#     train_size=.8,
#     random_state=SEED,
#     drop_columns=[]
# )

# X_train_Data = Data(X_train_Data)
# X_test_Data = Data(X_test_Data)

# <font color=#cd0000> Preprocesamiento </font>

## <font color=#cd0000> Eliminación de datos inválidos y valores atípicos </font>
- TODO: Breve descripción de qué es un dato inválido (-1's en columna, etc.)
- Eliminaremos aquellos valores fuera de los percentiles 5 y 95.
- TODO: Definiremos cuál será el límite de outliers permitido por serie temporal

In [ ]:
# TODO - Remove invalid data

In [ ]:
train.remove_outliers(
    headers=train.get_derived_data_columns()['attrs'],
    outliers_limit=.3
)

test.remove_outliers(
    headers=test.get_derived_data_columns()['attrs'],
    outliers_limit=.3
)


In [ ]:
import pandas as pd

# Remaining series
print("Train: Previous number of series: {}".format(
    len(pd.unique(train.original_data['id']))))
print("Train: Actual number of series: {}".format(
    len(pd.unique(train.derived_data['id']))))

print("Test: Previous number of series: {}".format(
    len(pd.unique(test.original_data['id']))))
print("Test: Actual number of series: {}".format(
    len(pd.unique(test.derived_data['id']))))


## <font color=#cd0000> Resoluciones a aplicar </font>
- TODO:
  - Si las series son rápidas (muchos cambios en poco tiempo) especificar resoluciones altas (sin modificaciones).
  - Si las series son lentas (pocos cambios en mucho tiempo) especificar resoluciones bajas (eliminamos datos).

In [ ]:
# Series lentas
train.reduce_sampling_rate(remove_one_each_n_samples=2)
test.reduce_sampling_rate(remove_one_each_n_samples=2)

# <font color=#cd0000> División en ventanas </font>
- Solo aplicaremos enventanado si no ha sido aplicado anteriormente
- TODO: Especificar tamaño de ventana esperado como mejor y adjuntar otro tamaño de ventana para comparar (al menos 2 más)
- TODO: No es necesario aplicar siempre el enventanado, revisar análisis en profundidad.

In [ ]:
# Estudiamos eventos globales (series lentas)
ws_x = train.get_shortest_serie().shape[0]
train_ws_x, train_windows_per_serie_x = \
    train.split_into_windows(train.derived_data, window_size=ws_x)
test_ws_x, test_windows_per_serie_x = \
    test.split_into_windows(test.derived_data, window_size=ws_x)

# Estudiamos eventos locales (series rápidas)
ws_y = int(train.get_shortest_serie().shape[0]/2)
train_ws_y, train_windows_per_serie_y =\
    train.split_into_windows(train.derived_data, window_size=ws_y)
test_ws_y, test_windows_per_serie_y =\
    test.split_into_windows(test.derived_data, window_size=ws_y)


In [ ]:
from utils.data_extraction import Data

train_large_windows = Data(train_ws_x, train_windows_per_serie_x)
test_large_windows = Data(test_ws_x, test_windows_per_serie_x)

train_short_windows = Data(train_ws_y, train_windows_per_serie_y)
test_short_windows = Data(test_ws_y, test_windows_per_serie_y)


## <font color=#cd0000> Codificación </font>

In [ ]:
from utils.codifications import temporal_trend_fn

train_large_windows.apply_codifications([temporal_trend_fn])
test_large_windows.apply_codifications([temporal_trend_fn])

# <font color=#cd0000> Preparación de los datos </font>

In [ ]:
X_train = train.derived_data.drop(['id', 'class'], axis=1)
X_test = test.derived_data.drop(['id', 'class'], axis=1)

y_train = train.derived_data['class'].to_numpy()
y_test = test.derived_data['class'].to_numpy()

## <font color=#cd0000> Técnicas de balanceo </font>

### <font color=#cd0000> Asignación de pesos a las clases </font>

In [ ]:
import numpy as np
from sklearn.utils import compute_class_weight

class_weights = compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train)
class_weights = {'abnormal': class_weights[0], 'normal': class_weights[1]}


# <font color=#cd0000> Diseño de la topología del bosque </font>
- Número de estimadores inicial recomendado
- Profundidad máxima recomendada

## <font color=#cd0000> Entrenamiento </font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=SEED)
clf.fit(X_train, y_train)

## <font color=#cd0000> Clasificación </font>

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
y_true = np.asarray(y_test)
    
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, zero_division=0))


## <font color=#cd0000> Discusión de los resultados </font>
- Vamos a estudiar diferentes rangos de hiper-parámetros interesantes que podrían darnos alguna pista sobre el rango en el que buscar el mejor clasificador de este tipo:
  - Para un `n_estimators` pequeño (10) y una `max_depth` pequeño (10) observamos resultados muy malos en lo que a la capacidad de generalización del clasificador se refiere (clasifica muy bien, únicamente, ejemplos de la clase `abnormal`):
    ```
    Confusion matrix:
    [[55337  2435]
     [20110  2514]]

    Classification report:
                  precision    recall  f1-score   support

        abnormal       0.73      0.96      0.83     57772
          normal       0.51      0.11      0.18     22624

        accuracy                           0.72     80396
       macro avg       0.62      0.53      0.51     80396
    weighted avg       0.67      0.72      0.65     80396
    ```

## <font color=#cd0000> Conclusiones </font>
- Como podemos observar ...
- No obstante si no tuviéramos más remedio que utilizarlo de esta forma buscaríamos el mejor en el rango orientativo:
  - `n_estimators`: [...]
  - `max_depth`: [...]

# <font color=#cd0000> Randomized Search </font>
- Búsqueda de hiper-parámetros aleatoria con RF maximizando ``macro avg f1-score``

## <font color=#cd0000> Rangos de búsqueda </font>
- Como vimos anteriormente los rangos de búsqueda aleatoria de los mejores hiper-parámetros serán los siguientes

In [ ]:
N_ESTIMATORS_RANGE = TODO
MAX_DEPTH_RANGE = TODO

In [ ]:
import random
import pickle
import utils.constants as cs
from sklearn.ensemble import RandomForestClassifier
from utils.classifier_utils import (windowed_cross_val,
                                    compute_classification_reports_means)
from utils.plot_utils import pretty_print_classification_report_dict


def rf_randomized_search_cv(
        windowed_series,
        relation_with_series,
        prefix,
        class_weights,
        cv=5):
    global PKL_DIR
    all_clf_used = {}

    n_samples = 5
    n_estimators_list = random.sample(list(N_ESTIMATORS_RANGE), n_samples)
    max_depth_list = random.sample(list(MAX_DEPTH_RANGE), n_samples)

    best_hyp_params = None
    best_score = 0
    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            clf = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                random_state=SEED,
                class_weight=class_weights
            )

            reports = windowed_cross_val(
                clf,
                windowed_series,
                relation_with_series,
                estimator_type=cs.ESTIMATOR_SKLEARN,
                cv=cv,
                drop_columns=['id', 'class'],
                seed=SEED
            )
            mean_report = compute_classification_reports_means(reports)
            all_clf_used[(n_estimators, max_depth)] = mean_report

            if mean_report['macro avg']['f1-score'][0] >= best_score:
                best_score = mean_report['macro avg']['f1-score'][0]
                best_hyp_params = (n_estimators, max_depth)
                best_report = mean_report

            print("\t\t--------------ACTUAL BEST: N_Estimators={}; Max_Depth={}--------------"
                  .format(best_hyp_params[0], best_hyp_params[1]))
            pretty_print_classification_report_dict(best_report)
            print("\t\t--------------ITERATION: N_Estimators={}; Max_Depth={}--------------"
                  .format(n_estimators, max_depth))
            pretty_print_classification_report_dict(mean_report)

    with open(PKL_DIR + prefix, 'wb') as file:
        pickle.dump(all_clf_used, file)

    return best_hyp_params, best_report


In [ ]:
rf_randomized_search_cv(
    train.derived_data,
    train.derived_data_windows_per_serie,
    PKL_NAME,
    class_weights,
    cv=5)


# <font color=#cd0000> Randomized Search con múltiples ejecuciones en lugar de Validación Cruzada </font>
- Solo si tenemos pocos datos
- Ejecutaremos el mismo modelo sobre diferentes particiones del conjunto de datos original para observar su desempeño.

In [ ]:
# TODO

# <font color=#cd0000> Análisis de resultados </font>
- Según la búsqueda aleatoria de hiper-parámetros, la mejor combinación, es la de ``n_estimators`` = TODO y ``max_depth`` = TODO:
    ```
        TODO
    ```
- Ahora vamos a visualizar la evolución de los resultados (25 resultados) para observar cómo avanza nuestra métrica objetivo -> Macro Average F1-Score.

In [ ]:
import pickle

all_reports = pickle.load(open(PKL_DIR + PKL_NAME, 'rb'))


In [ ]:
from utils.plot_utils import plot_score

macro_avg_f1_scores = dict(map(
    lambda z: (z, {'score': all_reports[z]['macro avg']['f1-score'][0],
                   'std': all_reports[z]['macro avg']['f1-score'][1]}),
    all_reports
))

plot_score(
    [macro_avg_f1_scores],
    ('n_estimators', 'max_depth'),
    'RandomForest',
    inverse=False,
    mode='score',
    in_same_graphic=True,
    accuracy_mode='accuracy',
    metric_name='Macro Average F1-Score'
)


In [ ]:
abnormal_recall_scores = dict(map(
    lambda z: (z, {'score': all_reports[z]['abnormal']['recall'][0],
                   'std': all_reports[z]['abnormal']['recall'][1]}),
    all_reports
))

normal_recall_scores = dict(map(
    lambda z: (z, {'score': all_reports[z]['normal']['recall'][0],
                   'std': all_reports[z]['normal']['recall'][1]}),
    all_reports
))

plot_score(
    [abnormal_recall_scores],
    ('n_estimators', 'max_depth'),
    'RandomForest',
    inverse=False,
    mode='score',
    in_same_graphic=True,
    accuracy_mode='accuracy',
    metric_name='Abnormal recall score'
)

plot_score(
    [normal_recall_scores],
    ('n_estimators', 'max_depth'),
    'RandomForest',
    inverse=False,
    mode='score',
    in_same_graphic=True,
    accuracy_mode='accuracy',
    metric_name='Normal recall score'
)


## <font color=#cd0000> Evaluación sobre el conjunto de validación </font>
- Vamos a llevar a cabo la evaluación final sobre el conjunto de validación (esto es lo que irá al apartado de ``Pruebas y Resultados`` de la memoria).

### <font color=#cd0000> Entrenamiento </font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=TODO,
    max_depth=TODO,
    class_weight=class_weights,
    random_state=SEED
)
clf.fit(X_train, y_train)


### <font color=#cd0000> Clasificación </font>

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
y_true = np.asarray(y_test)
    
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, zero_division=0))


# <font color=#cd0000> Conclusiones </font>
- TODO - Unas breves conclusiones sobre los resultados obtenidos (influencia de la codificación, ...)